# Chapter 46: Latest TypeScript Features

---

## Introduction

TypeScript continues to evolve rapidly, with version 5.x introducing significant improvements that enhance type safety, developer experience, and alignment with modern JavaScript standards. This chapter explores the cutting-edge features available in TypeScript 5.0 and beyond, including the standardized decorator proposal (Stage 3), the powerful `satisfies` operator, `const` type parameters for literal inference, and modern module resolution strategies.

These features represent the future of TypeScript development, addressing long-standing pain points like decorator standardization, type widening prevention, and ESM/CJS interoperability. Understanding these capabilities allows you to write more expressive, performant, and standards-compliant code while maintaining backward compatibility where necessary.

---

## 46.1 Decorators (Stage 3)

TypeScript 5.0 introduced support for the TC39 Stage 3 Decorators proposal, a significant departure from the legacy experimental decorators that have been available since TypeScript 1.5. The new standard is aligned with JavaScript's future and offers improved type inference and metadata capabilities.

### 46.1.1 Legacy vs. Standard Decorators

```typescript
// ❌ Legacy Experimental Decorators (pre-5.0)
// tsconfig.json: "experimentalDecorators": true, "emitDecoratorMetadata": true

function legacyLogged(target: any, propertyKey: string, descriptor: PropertyDescriptor) {
  const original = descriptor.value;
  descriptor.value = function(...args: any[]) {
    console.log(`Calling ${propertyKey}`);
    return original.apply(this, args);
  };
}

class LegacyExample {
  @legacyLogged
  greet() {
    console.log('Hello');
  }
}

// ✅ Standard Stage 3 Decorators (TypeScript 5.0+)
// tsconfig.json: No special flags needed (or "experimentalDecorators": false)

function logged(originalMethod: any, context: ClassMethodDecoratorContext) {
  const methodName = String(context.name);
  
  function replacementMethod(this: any, ...args: any[]) {
    console.log(`Entering ${methodName}`);
    const result = originalMethod.call(this, ...args);
    console.log(`Exiting ${methodName}`);
    return result;
  }
  
  return replacementMethod;
}

class ModernExample {
  @logged
  greet() {
    console.log('Hello');
  }
}
```

**Explanation:**
- Legacy decorators used the `experimentalDecorators` flag and followed an older TypeScript-specific implementation
- Standard decorators align with the TC39 proposal and will be the native JavaScript behavior
- The new API provides a `context` object containing metadata about the decorated element
- Standard decorators are more predictable and have better tooling support

### 46.1.2 Decorator Context and Types

```typescript
// Type-safe decorators with proper typing
type Constructor = new (...args: any[]) => {};

// Class decorator with context
function sealed<T extends Constructor>(target: T, context: ClassDecoratorContext<T>) {
  return class extends target {
    constructor(...args: any[]) {
      super(...args);
      Object.freeze(this);
    }
  };
}

// Method decorator with proper typing
function throttle<T extends (...args: any[]) => any>(
  target: T,
  context: ClassMethodDecoratorContext<object, T>
) {
  let lastExecution = 0;
  const interval = 1000; // 1 second
  
  function replacement(this: any, ...args: Parameters<T>): ReturnType<T> {
    const now = Date.now();
    if (now - lastExecution >= interval) {
      lastExecution = now;
      return target.apply(this, args);
    }
    return undefined as ReturnType<T>;
  }
  
  return replacement as T;
}

// Accessor decorator
function loggedAccessor<T>(
  target: ClassAccessorDecoratorTarget<object, T>,
  context: ClassAccessorDecoratorContext<object, T>
): ClassAccessorDecoratorResult<object, T> {
  return {
    get() {
      console.log(`Getting ${context.name.toString()}`);
      return target.get.call(this);
    },
    set(value) {
      console.log(`Setting ${context.name.toString()} to ${value}`);
      target.set.call(this, value);
    }
  };
}

// Field decorator
function required<T>(
  target: undefined,
  context: ClassFieldDecoratorContext<object, T>
) {
  return function(initialValue: T): T {
    if (initialValue === undefined || initialValue === null) {
      throw new Error(`${context.name.toString()} is required`);
    }
    return initialValue;
  }(target as T);
}

@sealed
class User {
  @required
  name: string = '';
  
  @loggedAccessor
  accessor age: number = 0;
  
  @throttle
  save() {
    console.log('Saving user...');
  }
}
```

**Explanation:**
- `ClassDecoratorContext`, `ClassMethodDecoratorContext`, etc., provide type-safe metadata
- Context includes `name`, `private`, `static`, and `addInitializer` for registration
- Decorators can return a replacement value or undefined to leave as-is
- Type parameters preserve the original method signatures when wrapping

### 46.1.3 Decorator Metadata (ES2023)

```typescript
// Using Symbol.metadata for decorator metadata (TypeScript 5.2+)
const VALIDATION_KEY = Symbol('validation');

interface ValidationRule {
  type: 'string' | 'number' | 'email';
  required?: boolean;
  minLength?: number;
}

function validate(rule: ValidationRule) {
  return function<T>(
    target: undefined,
    context: ClassFieldDecoratorContext<object, T>
  ) {
    context.addInitializer(function(this: any) {
      if (!this[Symbol.metadata]) {
        this[Symbol.metadata] = {};
      }
      if (!this[Symbol.metadata][VALIDATION_KEY]) {
        this[Symbol.metadata][VALIDATION_KEY] = {};
      }
      this[Symbol.metadata][VALIDATION_KEY][context.name] = rule;
    });
  };
}

class Product {
  @validate({ type: 'string', required: true, minLength: 3 })
  name: string = '';
  
  @validate({ type: 'number', required: true })
  price: number = 0;
  
  validate() {
    const rules = this[Symbol.metadata]?.[VALIDATION_KEY] || {};
    for (const [field, rule] of Object.entries(rules)) {
      const value = (this as any)[field];
      if (rule.required && !value) {
        throw new Error(`${field} is required`);
      }
      if (rule.minLength && value.length < rule.minLength) {
        throw new Error(`${field} must be at least ${rule.minLength} characters`);
      }
    }
  }
}
```

**Explanation:**
- `Symbol.metadata` provides a standardized way to attach metadata to classes
- `context.addInitializer` registers setup code that runs during class construction
- Metadata persists on the class and instances, enabling validation frameworks
- This replaces the legacy `Reflect.metadata` approach with a web standard

---

## 46.2 `const` Type Parameters

TypeScript 5.0 introduced `const` type parameters, allowing generic functions to infer literal types as if the arguments had `as const` applied, without requiring the caller to use that syntax.

### 46.2.1 The Problem of Type Widening

```typescript
// Without const type parameters
function createRoutes<T extends string[]>(paths: T): T {
  return paths;
}

// Inferred as string[] - literals are widened
const routes = createRoutes(['/home', '/about', '/contact']);
//    ^? const routes: string[]

// To preserve literals, caller must use as const (cumbersome)
const routesFixed = createRoutes(['/home', '/about', '/contact'] as const);
//    ^? const routesFixed: readonly ["/home", "/about", "/contact"]
```

### 46.2.2 Using `const` Type Parameters

```typescript
// ✅ Using const type parameter (TypeScript 5.0+)
function createRoutes<const T extends readonly string[]>(paths: T): T {
  return paths;
}

// Now literals are preserved automatically
const routes = createRoutes(['/home', '/about', '/contact']);
//    ^? const routes: readonly ["/home", "/about", "/contact"]

// Specific literal types inferred
const config = createRoutes([
  { path: '/home', component: 'Home' },
  { path: '/about', component: 'About' }
]);
//    ^? const config: readonly [{ readonly path: "/home"; readonly component: "Home"; }, { readonly path: "/about"; readonly component: "About"; }]
```

**Explanation:**
- The `const` modifier before the type parameter name tells TypeScript to infer the most specific type possible
- Equivalent to the caller adding `as const`, but built into the function signature
- Particularly useful for configuration objects, route definitions, and design systems

### 46.2.3 Practical Applications

```typescript
// Type-safe event emitter with const type parameters
type EventMap = {
  user.created: { userId: string; name: string };
  order.completed: { orderId: string; amount: number };
};

class TypedEmitter<const Events extends Record<string, any>> {
  private listeners = new Map<string, Set<Function>>();
  
  on<const K extends keyof Events>(
    event: K,
    handler: (data: Events[K]) => void
  ) {
    if (!this.listeners.has(event as string)) {
      this.listeners.set(event as string, new Set());
    }
    this.listeners.get(event as string)!.add(handler);
  }
  
  emit<const K extends keyof Events>(event: K, data: Events[K]) {
    this.listeners.get(event as string)?.forEach(fn => fn(data));
  }
}

// Usage - types are inferred precisely
const emitter = new TypedEmitter<EventMap>();

emitter.on('user.created', (data) => {
  // data is inferred as { userId: string; name: string }
  console.log(data.userId);
});

// Error: Argument of type '"invalid"' is not assignable to parameter of type '"user.created" | "order.completed"'
// emitter.on('invalid', () => {});

// Type-safe color palette generator
function createPalette<const T extends Record<string, string>>(colors: T): T {
  return colors;
}

const palette = createPalette({
  primary: '#007bff',
  secondary: '#6c757d',
  success: '#28a745'
});

// palette.primary is literally '#007bff', not string
// Enables autocomplete for specific color names
type ColorKey = keyof typeof palette; // "primary" | "secondary" | "success"
```

---

## 46.3 The `satisfies` Operator

Introduced in TypeScript 4.9, the `satisfies` operator allows you to validate that an expression matches a type without widening the expression's type to that type. This preserves the specific literal types while ensuring structural constraints are met.

### 46.3.1 The Problem: `as` vs. Type Annotations

```typescript
// ❌ Type annotation widens the type
const config: Record<string, string> = {
  host: 'localhost',
  port: '3000'
};
config.port; // string (lost the literal '3000')

// ❌ 'as const' is too strict for validation
const configConst = {
  host: 'localhost',
  port: 3000  // Error if we try to validate against Record<string, string>
} as const;

// ❌ Type assertion bypasses checking
const configAssert = {
  host: 'localhost',
  port: 3000 
} as Record<string, string>; // No error, but port is number not string!
```

### 46.3.2 Using `satisfies`

```typescript
// ✅ satisfies validates without widening
const config = {
  host: 'localhost',
  port: '3000'
} satisfies Record<string, string>;

// config.host is 'localhost' (literal preserved)
// config.port is '3000' (literal preserved)
// But we know it satisfies Record<string, string>

// Practical example: Validation with preserved types
type RouteConfig = {
  path: string;
  component: string;
  exact?: boolean;
};

const routes = [
  { path: '/home', component: 'Home', exact: true },
  { path: '/about', component: 'About' }
] satisfies RouteConfig[];

// routes[0].path is '/home' (literal), not string
// But validated against RouteConfig structure

// Catching excess properties while preserving literals
type CSSColor = Record<string, `#${string}`>;

const theme = {
  primary: '#007bff',
  secondary: '#6c757d',
  // invalid: 'red' // Error: Type 'string' is not assignable to type '`#${string}`'
} satisfies CSSColor;

// theme.primary is '#007bff', not just string
```

**Explanation:**
- `satisfies` checks that the value conforms to the type constraint
- Unlike type annotations (`: Type`), it doesn't widen the inferred type
- Unlike `as` assertions, it actually validates the type structure
- Perfect for configuration objects where you want autocomplete and validation but need literal types for discriminated unions

### 46.3.3 Use Cases and Patterns

```typescript
// Validating object keys while preserving literal union
type EventName = 'click' | 'hover' | 'scroll';

const handlers = {
  click: (e: MouseEvent) => console.log('clicked'),
  hover: (e: MouseEvent) => console.log('hovered'),
  // scroll: (e: WheelEvent) => {} // Error if missing required keys
} satisfies Record<EventName, (e: MouseEvent) => void>;

// handlers is Record<"click" | "hover" | "scroll", ...>
// Not just Record<EventName, ...> - preserves specific keys

// Ensuring complete switch case handling
type Status = 'loading' | 'success' | 'error';

const statusMessages = {
  loading: 'Loading...',
  success: 'Done!',
  error: 'Failed'
} satisfies Record<Status, string>;

// If we add 'pending' to Status type, we get an error here
// because statusMessages doesn't satisfy the new type

// API response validation with preserved types
type ApiResponse<T> = {
  data: T;
  status: number;
};

const response = {
  data: { id: 1, name: 'John' },
  status: 200,
  timestamp: Date.now() // Extra property allowed (structural typing)
} satisfies ApiResponse<{ id: number; name: string }>;

// response.data.id is number, but we know it satisfies the API contract
```

---

## 46.4 Import Attributes and Assertions

TypeScript 4.5+ supports import assertions (now import attributes in newer proposals) for importing non-JavaScript modules like JSON with type safety.

### 46.4.1 Import Assertions Syntax

```typescript
// Importing JSON with type assertion (TypeScript 4.5-5.2)
import config from './config.json' assert { type: 'json' };

// config is typed based on the JSON structure
// TypeScript analyzes the JSON file and infers:
// typeof config = { name: string; version: string; ... }

// Dynamic import with assertion
const data = await import('./data.json', {
  assert: { type: 'json' }
});

// Usage with moduleResolution: bundler (TypeScript 4.7+)
import pkg from './package.json' assert { type: 'json' };
console.log(pkg.version); // Fully typed
```

### 46.4.2 Import Attributes (ES2023/TypeScript 5.3+)

```typescript
// New import attributes syntax (replaces assert)
import config from './config.json' with { type: 'json' };

// Dynamic import with attributes
const module = await import('./module.wasm', {
  with: { type: 'webassembly' }
});

// TypeScript validates the assertion
// import bad from './config.json' assert { type: 'css' };
// Error: Type '"css"' is not assignable to type '"json"'

// Multiple attributes
import styles from './styles.css' with { type: 'css', integrity: 'sha256-abc123' };
```

**Explanation:**
- Import assertions ensure the runtime treats the import correctly (e.g., as JSON not JS)
- TypeScript uses these to load and type-check JSON files at compile time
- The `with` syntax is the newer standard replacing `assert`
- Requires `moduleResolution` to be `node16`, `nodenext`, or `bundler`

### 46.4.3 Typing JSON Imports

```typescript
// Declare JSON module types for specific files
declare module './config.json' {
  interface Config {
    apiUrl: string;
    timeout: number;
    features: {
      darkMode: boolean;
      betaAccess: boolean;
    };
  }
  const value: Config;
  export default value;
}

// Now imports are specifically typed
import config from './config.json' with { type: 'json' };
// config is Config interface, not just any

// Generic JSON type declaration (fallback)
declare module '*.json' {
  const value: any;
  export default value;
}
```

---

## 46.5 Enum Improvements and Modern Alternatives

TypeScript 5.0+ brings improvements to enum handling, while modern TypeScript encourages const objects with `as const` as alternatives to traditional enums.

### 46.5.1 Modern Enum Behavior

```typescript
// TypeScript 5.0+ stricter enum comparison checks
enum Status {
  Active = 'active',
  Inactive = 'inactive'
}

// Stricter literal comparison
const s: Status = Status.Active;

// This comparison now works correctly with string enums
if (s === 'active') { // OK in TypeScript 5.0+
  console.log('Active');
}

// Numeric enum issues remain (avoid when possible)
enum Priority {
  Low,    // 0
  Medium, // 1
  High    // 2
}

// Type safety hole: numeric enums accept any number
const p: Priority = 99; // No error! Dangerous.
```

### 46.5.2 Const Objects vs. Enums (Recommended)

```typescript
// ✅ Modern alternative: const object with as const
const Status = {
  Active: 'active',
  Inactive: 'inactive',
  Pending: 'pending'
} as const;

// Extract type
type Status = typeof Status[keyof typeof Status];
// ^? type Status = "active" | "inactive" | "pending"

// Usage benefits:
// 1. No generated code (enums generate IIFEs)
// 2. True type safety (no numeric holes)
// 3. Works with satisfies for validation
const UserStatus = {
  Active: 'active',
  Inactive: 'inactive'
} as const satisfies Record<string, string>;

// Reverse mapping without enums
const StatusReverse = {
  active: 'Active',
  inactive: 'Inactive'
} as const satisfies Record<typeof Status[keyof typeof Status], string>;

// Helper type for reverse mapping
type StatusKey = keyof typeof Status; // "Active" | "Inactive" | "Pending"
```

**Explanation:**
- Const objects with `as const` provide the same autocomplete as enums
- No runtime code generation (enums emit JavaScript objects/functions)
- Better tree-shaking in bundlers
- No unexpected numeric coercion issues
- `satisfies` ensures the object meets your contract while preserving literals

### 46.5.3 Const Enum Considerations

```typescript
// const enums are inlined at compile time (no runtime object)
const enum HttpStatus {
  OK = 200,
  NotFound = 404,
  Error = 500
}

const status = HttpStatus.OK; // Compiles to: const status = 200;

// ⚠️ Pitfall: const enums cannot be used with isolatedModules or Babel
// isolatedModules: true (required for modern bundlers) breaks const enums
// across module boundaries

// ✅ Solution: Use regular enums or const objects for library code
export enum HttpStatusRegular {
  OK = 200,
  NotFound = 404
}

// Or use the const object pattern for guaranteed inlining
export const HttpStatus = {
  OK: 200,
  NotFound: 404
} as const;
export type HttpStatus = typeof HttpStatus[keyof typeof HttpStatus];
```

---

## 46.6 Module Resolution Enhancements

TypeScript 4.7+ introduced `moduleResolution: "bundler"` and enhanced support for modern package.json exports, improving compatibility with Vite, Webpack, and ES modules.

### 46.6.1 The `bundler` Module Resolution

```typescript
// tsconfig.json
{
  "compilerOptions": {
    "module": "ESNext",
    "moduleResolution": "bundler", // New in TypeScript 4.7+
    "target": "ES2022",
    "esModuleInterop": true
  }
}
```

**Why `bundler` mode:**
- Allows importing packages that lack default exports without `esModuleInterop` workarounds
- Supports the `exports` field in package.json more comprehensively
- Allows extensionless imports (like `import './file'` instead of `import './file.js'`)
- Better alignment with how modern bundlers (Vite, Webpack 5, esbuild) resolve modules

### 46.6.2 Package.json Exports Support

```typescript
// Modern library package.json with exports
{
  "name": "@mycompany/utils",
  "exports": {
    ".": {
      "types": "./dist/index.d.ts",
      "import": "./dist/index.mjs",
      "require": "./dist/index.cjs",
      "default": "./dist/index.js"
    },
    "./math": {
      "types": "./dist/math.d.ts",
      "import": "./dist/math.mjs"
    },
    "./package.json": "./package.json"
  }
}

// TypeScript now correctly resolves these subpaths
import { something } from '@mycompany/utils';        // Main export
import { add } from '@mycompany/utils/math';         // Subpath export (TypeScript 4.7+)

// With moduleResolution: bundler or node16/nodenext
```

**Explanation:**
- The `exports` field allows packages to define multiple entry points
- TypeScript 5.x can resolve `types` conditions in exports for accurate type definitions
- `moduleResolution: bundler` supports extensionless imports common in bundler ecosystems
- `node16` and `nodenext` are stricter, requiring full relative paths with extensions

### 46.6.3 ESM/CJS Interoperability

```typescript
// TypeScript 5.2+ improved ESM/CJS detection
// File: utils.mts (explicit ES module)
export const foo = 'bar';

// File: utils.cts (explicit CommonJS)
exports.foo = 'bar';

// TypeScript enforces correct syntax per extension:
// .mts files must use ES modules syntax (import/export)
// .cts files must use CommonJS syntax (require/exports)

// Conditional exports in package.json
{
  "type": "module",
  "exports": {
    ".": {
      "import": {
        "types": "./dist/index.d.mts",
        "default": "./dist/index.mjs"
      },
      "require": {
        "types": "./dist/index.d.cts",
        "default": "./dist/index.cjs"
      }
    }
  }
}
```

---

## 46.7 Performance Improvements

TypeScript 5.0+ features significant architectural improvements, including a rewrite of the type checker and build system for faster compilation.

### 46.7.1 Compiler Performance Enhancements

```typescript
// TypeScript 5.0 is ~10-20% faster than 4.x due to:
// - Module resolution caching
// - Optimized type checking algorithms
// - Reduced memory allocations

// tsconfig.json optimizations for large projects
{
  "compilerOptions": {
    // Enable incremental compilation
    "incremental": true,
    "tsBuildInfoFile": "./.tsbuildinfo",
    
    // Skip type checking of declaration files (huge speedup)
    "skipLibCheck": true,
    
    // Skip default lib type checking
    "skipDefaultLibCheck": true,
    
    // Project references for monorepos (parallel builds)
    "composite": true,
    
    // Isolated modules (required for modern bundlers, faster checks)
    "isolatedModules": true
  }
}
```

### 46.7.2 Go-to-Definition and IDE Performance

```typescript
// TypeScript 5.3+ improved handling of complex types
// Better support for:
// - Large union types (discriminated unions)
// - Complex mapped types
// - Recursive type definitions

// Pattern for optimal IDE performance:
// Prefer interfaces over complex type aliases for public APIs
interface User {
  name: string;
  settings: UserSettings;
}

interface UserSettings {
  theme: 'light' | 'dark';
}

// Avoid deeply nested conditional types in public signatures
// ❌ Slow in IDE:
type ComplexTransform<T> = T extends object 
  ? { [K in keyof T]: T[K] extends string ? Uppercase<T[K]> : T[K] }
  : never;

// ✅ Faster: Use helper types or interfaces
type StringKeys<T> = {
  [K in keyof T]: T[K] extends string ? K : never
}[keyof T];
```

### 46.7.3 Memory Optimization

```typescript
// For extremely large codebases (100k+ files):
// Use project references to split compilation units

// tsconfig.base.json
{
  "compilerOptions": {
    "strict": true,
    "composite": true
  }
}

// packages/shared/tsconfig.json
{
  "extends": "../../tsconfig.base.json",
  "compilerOptions": {
    "outDir": "./dist",
    "rootDir": "./src"
  },
  "include": ["src/**/*"]
}

// packages/app/tsconfig.json
{
  "extends": "../../tsconfig.base.json",
  "references": [
    { "path": "../shared" }
  ],
  "compilerOptions": {
    "outDir": "./dist"
  }
}

// Build with: tsc --build (caches each project separately)
```

---

## 46.8 Chapter Summary and Exercises

### Chapter Summary

This chapter covered the latest TypeScript 5.x capabilities:

1. **Standard Decorators (Stage 3)**: The new TC39-compliant decorator proposal with `context` objects, supporting classes, methods, accessors, fields, and metadata via `Symbol.metadata`.

2. **`const` Type Parameters**: Generic functions can now infer literal types without requiring callers to use `as const`, improving developer experience for configuration functions and type-safe event emitters.

3. **`satisfies` Operator**: Validates type constraints without widening, preserving specific literal types while ensuring structural correctness—ideal for configuration objects and API contracts.

4. **Import Attributes**: Modern syntax (`with { type: 'json' }`) for importing non-JavaScript modules, replacing the older `assert` syntax and providing compile-time validation of import types.

5. **Enum Modernization**: Const objects with `as const` are now preferred over traditional enums for better type safety, tree-shaking, and no runtime code generation.

6. **Module Resolution**: The `bundler` mode aligns TypeScript with modern build tools, while improved `exports` field support enables proper subpath imports with type definitions.

7. **Performance**: TypeScript 5.0+ offers 10-20% faster compilation, better memory management, and optimized IDE features through architectural improvements.

### Feature Adoption Checklist

```
┌─────────────────────────────────────────────────────────────────────┐
│                    TypeScript 5.x Feature Adoption                   │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   Immediate Adoption (Safe & Beneficial)                            │
│   □ Use satisfies for configuration objects                         │
│   □ Adopt const type parameters for utility functions               │
│   □ Switch to moduleResolution: bundler (if using Vite/Webpack)     │
│   □ Use const objects instead of enums for new code                 │
│                                                                     │
│   Gradual Migration                                                 │
│   □ Migrate experimental decorators to Stage 3 (breaking change)    │
│   □ Update import assertions to import attributes                   │
│   □ Adopt .mts/.cts extensions for explicit module types            │
│                                                                     │
│   Infrastructure                                                    │
│   □ Update to TypeScript 5.0+ for build performance                 │
│   □ Enable isolatedModules for faster checks                        │
│   □ Use project references for monorepo performance                 │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### Practical Exercises

**Exercise 1: Standard Decorators**

Convert this legacy experimental decorator to the new Stage 3 standard:

```typescript
// Legacy (experimental)
function measure(target: any, propertyKey: string, descriptor: PropertyDescriptor) {
  const original = descriptor.value;
  descriptor.value = async function(...args: any[]) {
    const start = performance.now();
    const result = await original.apply(this, args);
    console.log(`${propertyKey} took ${performance.now() - start}ms`);
    return result;
  };
}

// Tasks:
// 1. Convert to standard decorator syntax (TypeScript 5.0+)
// 2. Add proper typing for async methods
// 3. Ensure the decorator preserves the return type
// 4. Add metadata tracking using Symbol.metadata
```

**Exercise 2: Const Type Parameters**

Create a type-safe event system using const type parameters:

```typescript
// Create an EventBus class that:
// 1. Uses const type parameters to preserve event names as literals
// 2. Maps event names to specific payload types
// 3. Provides autocomplete for event names and typed payloads
// 4. Validates that all handlers are called with correct data

const eventBus = new EventBus({
  'user.login': { userId: string; timestamp: number },
  'user.logout': { userId: string; reason?: string }
});

// Should infer literals:
eventBus.on('user.login', (data) => {
  // data should be { userId: string; timestamp: number }
});
```

**Exercise 3: Satisfies Operator**

Refactor this configuration using `satisfies`:

```typescript
// Before: Using type annotation (widens types)
const theme: Record<string, string> = {
  primary: '#007bff',
  secondary: '#6c757d',
  success: '#28a745'
};

// Tasks:
// 1. Use satisfies to ensure it matches Record<string, `#${string}`>
// 2. Ensure theme.primary is literally '#007bff', not string
// 3. Add validation that no extra keys are added (via satisfies constraints)
// 4. Create a type ThemeConfig that enforces required color keys
```

**Exercise 4: Module Resolution**

Set up a modern TypeScript package structure:

```typescript
// Create a package.json with:
// 1. Exports for main entry and ./utils subpath
// 2. Separate type definitions for ESM and CJS
// 3. TypeScript config using moduleResolution: bundler
// 4. Demonstrate importing from both entry points with full type safety
```

**Exercise 5: Migration from Enums**

Refactor this enum to a modern const object approach:

```typescript
// Current enum
enum Permission {
  Read = 'read',
  Write = 'write',
  Admin = 'admin'
}

function checkPermission(user: User, perm: Permission): boolean {
  return user.permissions.includes(perm);
}

// Tasks:
// 1. Convert to const object with as const
// 2. Extract the Permission type
// 3. Ensure reverse mapping still works (Permission.Read -> 'read', and 'read' -> 'Read')
// 4. Add satisfies validation that all values are lowercase strings
// 5. Ensure no runtime code is generated (unlike enums)
```

---

## End of Handbook
 Now that we've explored the latest features in TypeScript 5.x, you have the tools to write more expressive, performant, and type-safe code. Whether you're adopting new decorators, leveraging const type parameters, or optimizing your module resolution strategy, these capabilities will help you build robust applications that align with modern JavaScript standards. Happy coding!

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../13. best_practices_and_guidelines/45. migration_strategies.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <span style='color:gray; font-size:1.05em;'>Next</span>
</div>
